In [1]:
!pip install mediapipe opencv-python


In [1]:
import ipywidgets as widgets
from IPython.display import display

camera_active = True

stop_button = widgets.Button(
    description='Stop Camera',
    button_style='danger',
    icon='stop'
)

def on_stop_button_clicked(b):
    global camera_active
    camera_active = False
    print("Camera stopped. Run the cell again to restart.")

stop_button.on_click(on_stop_button_clicked)
display(stop_button)


Button(button_style='danger', description='Stop Camera', icon='stop', style=ButtonStyle())

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def draw_rounded_rectangle(img, top_left, bottom_right, radius, color, thickness=-1):
    x1, y1 = top_left
    x2, y2 = bottom_right
    cv2.rectangle(img, (x1 + radius, y1), (x2 - radius, y1 + radius), color, thickness)
    cv2.rectangle(img, (x1, y1 + radius), (x2, y2 - radius), color, thickness)
    cv2.rectangle(img, (x1 + radius, y2 - radius), (x2 - radius, y2), color, thickness)
    cv2.circle(img, (x1 + radius, y1 + radius), radius, color, thickness)
    cv2.circle(img, (x2 - radius, y1 + radius), radius, color, thickness)
    cv2.circle(img, (x1 + radius, y2 - radius), radius, color, thickness)
    cv2.circle(img, (x2 - radius, y2 - radius), radius, color, thickness)

cap = cv2.VideoCapture(0)

counter = 0
stage = None
feedback = ""
baseline_distance = 0
baseline_set = False
calibration_frames = 0
calibration_sum = 0
tuck_threshold = 0.04  # Adjust as needed (percentage of frame height)
panel_width = 400
panel_height = 100
instruction_panel_height = 160
corner_radius = 20
bg_color = (30, 30, 30)
text_color = (255, 255, 255)
accent_color = (0, 200, 255)
progress_color = (0, 255, 200)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark
            # Nose
            nose = landmarks[mp_pose.PoseLandmark.NOSE.value]
            # Approximate chin as midpoint between mouth left and right
            mouth_left = landmarks[mp_pose.PoseLandmark.MOUTH_LEFT.value]
            mouth_right = landmarks[mp_pose.PoseLandmark.MOUTH_RIGHT.value]
            chin_x = (mouth_left.x + mouth_right.x) / 2
            chin_y = (mouth_left.y + mouth_right.y) / 2
            # Neck as midpoint between shoulders
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
            neck_x = (left_shoulder.x + right_shoulder.x) / 2
            neck_y = (left_shoulder.y + right_shoulder.y) / 2

            h, w, _ = image.shape
            # Distance from nose to neck (vertical)
            nose_neck_dist = abs((nose.y - neck_y))

            # Calibration
            if not baseline_set:
                if calibration_frames < 30:
                    calibration_sum += nose_neck_dist
                    calibration_frames += 1
                    feedback = f"Calibrating... {calibration_frames}/30"
                else:
                    baseline_distance = calibration_sum / calibration_frames
                    baseline_set = True
                    feedback = "Ready. Tuck your chin!"
            else:
                # Calculate change from baseline
                delta = baseline_distance - nose_neck_dist  # Positive when chin tucks in
                # Tuck detected
                if delta > tuck_threshold and (stage == 'neutral' or stage is None):
                    stage = 'tucked'
                    feedback = "Chin tucked - good!"
                # Return to neutral
                elif delta < tuck_threshold/2 and stage == 'tucked':
                    stage = 'neutral'
                    counter += 1
                    feedback = "Rep counted!"
                elif stage is None:
                    stage = 'neutral'
                    feedback = "Ready to start"
                # Progress gauge
                tuck_gauge = max(0, min(100, (delta / (tuck_threshold * 2)) * 100))
                if stage == 'tucked':
                    feedback = f"Hold tuck: {int(tuck_gauge)}%"
                elif stage == 'neutral' and counter > 0:
                    feedback = "Return to neutral"

        except Exception as e:
            pass

        # UI Panels
        h, w, _ = image.shape
        bottom_panel_x = (w - panel_width) // 2
        bottom_panel_y = h - panel_height - 20
        top_panel_x = (w - panel_width) // 2
        top_panel_y = 20

        # Instruction panel
        instruction_overlay = image.copy()
        draw_rounded_rectangle(
            instruction_overlay,
            (top_panel_x, top_panel_y),
            (top_panel_x + panel_width, top_panel_y + instruction_panel_height),
            corner_radius,
            bg_color,
            -1
        )
        alpha = 0.8
        cv2.addWeighted(instruction_overlay, alpha, image, 1 - alpha, 0, image)
        cv2.putText(image, "CHIN TUCK EXERCISE",
                    (top_panel_x + panel_width//2 - 110, top_panel_y + 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, accent_color, 1, cv2.LINE_AA)
        instructions = [
            "1. Sit/stand facing the camera, head neutral",
            "2. Gently tuck your chin toward your chest",
            "3. Hold briefly at the end position",
            "4. Return to neutral (look straight)",
            "5. Repeat for desired reps"
        ]
        line_spacing = 22
        start_y = top_panel_y + 60
        for i, instr in enumerate(instructions):
            y_pos = start_y + (i * line_spacing)
            cv2.putText(image, instr,
                        (top_panel_x + 25, y_pos),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.45, text_color, 1, cv2.LINE_AA)

        # Counter panel
        counter_overlay = image.copy()
        draw_rounded_rectangle(
            counter_overlay,
            (bottom_panel_x, bottom_panel_y),
            (bottom_panel_x + panel_width, bottom_panel_y + panel_height),
            corner_radius,
            bg_color,
            -1
        )
        cv2.addWeighted(counter_overlay, alpha, image, 1 - alpha, 0, image)

        # Progress gauge (vertical bar)
        gauge_width = 6
        gauge_x = bottom_panel_x + panel_width - 50
        gauge_height = panel_height - 40
        gauge_y = bottom_panel_y + 20
        cv2.rectangle(image,
                      (gauge_x, gauge_y),
                      (gauge_x + gauge_width, gauge_y + gauge_height),
                      (80, 80, 80), -1, cv2.LINE_AA)
        if baseline_set:
            try:
                delta = baseline_distance - nose_neck_dist
                tuck_gauge = max(0, min(100, (delta / (tuck_threshold * 2)) * 100))
                fill_height = int(gauge_height * (tuck_gauge / 100))
                cv2.rectangle(image,
                              (gauge_x, gauge_y + gauge_height - fill_height),
                              (gauge_x + gauge_width, gauge_y + gauge_height),
                              progress_color, -1, cv2.LINE_AA)
            except:
                pass
        cv2.putText(image, "TUCK",
                    (gauge_x - 10, gauge_y - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, text_color, 1, cv2.LINE_AA)
        cv2.putText(image, "NEUTRAL",
                    (gauge_x - 30, gauge_y + gauge_height + 15),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, text_color, 1, cv2.LINE_AA)

        # Add label and counter
        cv2.putText(image, "CHIN TUCKS",
                    (bottom_panel_x + panel_width//2 - 70, bottom_panel_y + 25),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, text_color, 1, cv2.LINE_AA)
        cv2.putText(image, str(counter),
                    (bottom_panel_x + panel_width//2 - (15 if counter < 10 else 25), bottom_panel_y + 55),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, text_color, 2, cv2.LINE_AA)
        if feedback:
            cv2.putText(image, feedback,
                        (bottom_panel_x + 20, bottom_panel_y + 85),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, accent_color, 1, cv2.LINE_AA)

        # Draw key landmarks for visualization
        if results.pose_landmarks:
            landmarks_overlay = image.copy()
            key_landmarks = [
                mp_pose.PoseLandmark.NOSE,
                mp_pose.PoseLandmark.MOUTH_LEFT,
                mp_pose.PoseLandmark.MOUTH_RIGHT,
                mp_pose.PoseLandmark.LEFT_SHOULDER,
                mp_pose.PoseLandmark.RIGHT_SHOULDER
            ]
            for landmark_id in key_landmarks:
                landmark = results.pose_landmarks.landmark[landmark_id.value]
                cx, cy = int(landmark.x * w), int(landmark.y * h)
                cv2.circle(landmarks_overlay, (cx, cy), 5, accent_color, -1)
            # Draw lines: nose to neck, mouth to mouth
            neck_cx, neck_cy = int(neck_x * w), int(neck_y * h)
            nose_cx, nose_cy = int(nose.x * w), int(nose.y * h)
            cv2.line(landmarks_overlay, (nose_cx, nose_cy), (neck_cx, neck_cy), progress_color, 2)
            mouth_l_cx, mouth_l_cy = int(mouth_left.x * w), int(mouth_left.y * h)
            mouth_r_cx, mouth_r_cy = int(mouth_right.x * w), int(mouth_right.y * h)
            cv2.line(landmarks_overlay, (mouth_l_cx, mouth_l_cy), (mouth_r_cx, mouth_r_cy), progress_color, 2)
            landmarks_alpha = 0.2
            cv2.addWeighted(landmarks_overlay, landmarks_alpha, image, 1 - landmarks_alpha, 0, image)

        cv2.imshow('Chin Tuck Tracker', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
